In [108]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

In [109]:
layer = 1
table = "tweets"
path = f"./layer{layer}/{table}"

In [110]:
tweets = spark.read.parquet(path)

In [111]:
tweets.count()

1497

In [112]:
list(tweets.schema)

[StructField(tweet_id,StringType,true),
 StructField(created_at,StringType,true),
 StructField(text,StringType,true),
 StructField(hashtags,StringType,true),
 StructField(retweet_count,IntegerType,true),
 StructField(possibly_sensitive,BooleanType,true),
 StructField(lang,StringType,true),
 StructField(user_id,StringType,true),
 StructField(user_name,StringType,true),
 StructField(user_description,StringType,true),
 StructField(user_verification,BooleanType,true),
 StructField(user_followers_count,IntegerType,true),
 StructField(user_friends_count,IntegerType,true),
 StructField(user_created_at,StringType,true),
 StructField(user_location,StringType,true),
 StructField(etl_load,TimestampType,true),
 StructField(etl_load_partition_year,IntegerType,true),
 StructField(etl_load_partition_month,IntegerType,true),
 StructField(etl_load_partition_day,IntegerType,true),
 StructField(etl_load_partition_hour,IntegerType,true)]

In [113]:
from datetime import datetime

In [159]:
fact_tweet = (
    tweets
    .withColumn("tmsp3", F.to_timestamp("tmsp2", "EEE MMM dd HH:mm:ss yyyy"))
    .withColumn("created_at_2", F.regexp_replace("created_at", "\+0000 ", ""))
    .withColumn("created_at_3", F.to_timestamp("created_at", "EEE MMM dd HH:mm:ss +xxxx yyyy"))
    .select(
        "tweet_id",    
        "tmsp",
        "tmsp2",
#         "tmsp3",
        "created_at",
#         "created_at_2",
        "created_at_3",
#         "text",    
#         "hashtags",    
#         "retweet_count",    
#         "possibly_sensitive",    
#         "lang",   
#         "user_id",
    )
)
fact_tweet.toPandas()

,tweet_id,tmsp,tmsp2,created_at,created_at_3
0,1358589255191248898,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 01:31:34 +0000 2021,NaT
1,1358589255782719490,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 01:31:34 +0000 2021,NaT
2,1358589256218906625,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 01:31:34 +0000 2021,NaT
3,1358589257548525571,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 01:31:35 +0000 2021,NaT
4,1358589259784028165,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 01:31:35 +0000 2021,NaT
...,...,...,...,...,...
1492,1358570146458193925,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 00:15:38 +0000 2021,NaT
1493,1358569997824573441,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 00:15:03 +0000 2021,NaT
1494,1358570001880518656,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 00:15:04 +0000 2021,NaT
1495,1358570005357617153,2021-02-08 01:31:34,Mon Feb 08 01:31:34 2021,Mon Feb 08 00:15:04 +0000 2021,NaT


In [29]:
dim_user = (
    tweets
    .select(
         "user_id",
         "user_name",
         "user_description",
         "user_verification",
         "user_followers_count",
         "user_friends_count",
         "user_created_at",
         "user_location",
    )
)